In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mpld3
import os
import torch
import torchaudio
import numpy as np

### Load wav files into tensors

In [2]:
!ls ../wavs

classical_music_wavs  vctk_wavs


In [8]:
wav_directory = os.path.abspath("../wavs/vctk_wavs")
wav_tensors = []
for i, filename in enumerate(os.listdir(wav_directory)):
    if i%100 == 0:
        print(i)
    if filename.endswith(".wav"):
        wav_tensor, sample_rate = torchaudio.load_wav(os.path.join(wav_directory, filename))
#         print(wav_tensor.shape, sample_rate)
        wav_tensors.append(wav_tensor)

KeyboardInterrupt: 

### Concatenate tensors

In [60]:
wav_tensor = torch.cat(wav_tensors, dim=1).squeeze()
wav_tensor.min(), wav_tensor.max()

(tensor(-32768.), tensor(32767.))

### Rescale wav_tensor to [-1, 1]

In [61]:
wav_tensor += 32768
wav_tensor /= wav_tensor.max()
wav_tensor *= 2
wav_tensor -= 1
wav_tensor.min(), wav_tensor.max()

(tensor(-1.), tensor(1.))

### Rescale wav_tensor to [256] with mu law

In [62]:
mu_law_encode = torchaudio.transforms.MuLawEncoding(quantization_channels=256)
mu_wav_tensor = mu_law_encode(wav_tensor)
mu_wav_tensor = mu_wav_tensor.type(torch.int)
mu_wav_tensor.min(), mu_wav_tensor.max()

(tensor(0, dtype=torch.int32), tensor(255, dtype=torch.int32))

### Save wav_tensor

In [63]:
torch.save(mu_wav_tensor, 'wav_audio/wav_tensor.pt')